In [1]:
# from huggingface_hub import snapshot_download

# snapshot_download(
#     repo_id="malaysia-ai/Multilingual-TTS", 
#     repo_type="dataset",
#     allow_patterns="*/*.parquet",
#     local_dir="./Multilingual-TTS",
# )

In [2]:
# snapshot_download(
#     repo_id="malaysia-ai/Multilingual-TTS", 
#     repo_type="dataset",
#     allow_patterns="*_neucodec.zip",
#     local_dir="./",
# )

In [3]:
from glob import glob
import pandas as pd
import json
import os
import torch
import IPython.display as ipd

torch.set_grad_enabled(False)

from transformers import AutoTokenizer, AddedToken
from streaming import MDSWriter
from streaming.base.format.mds.encodings import Encoding, _encodings
from streaming import LocalDataset
import numpy as np
from tqdm import tqdm
from multiprocess import Pool
import itertools

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)

def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))

class UInt32(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint32)

_encodings['uint32'] = UInt32

columns = {
    'input_ids': 'uint32',
    'position_ids': 'uint32',
    'attention_mask': 'uint32',
    'audio': 'str',
    'text': 'str'
}
hashes = 'sha1', 'xxh64'

def new_path(f):
    splitted = f.split('/')
    folder = f.split('/')[0]
    folder = folder + '_neucodec'
    new_f = os.path.join(folder, '/'.join(splitted[1:]))
    new_f = new_f.replace('.mp3', '.json').replace('.wav', '.json')
    return new_f

/root/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [ ]:
def loop(files):
    files, _ = files
    data = []
    for f in tqdm(files):
        df = pd.read_parquet(f).to_dict(orient = 'records')
        for i in range(len(df)):
            token_filename = new_path(df[i]['audio_filename'])
            if not os.path.exists(token_filename):
                continue
            df[i]['token_filename'] = token_filename
            data.append(df[i])
    return data

In [ ]:
files = glob('Multilingual-TTS/*/*.parquet')
files = [f for f in files if 'Malaysian-TTS-v2' not in f]
data = multiprocessing(files, loop, cores = 20)
len(data)

In [ ]:
malaysian = pd.read_parquet('Multilingual-TTS/Malaysian-TTS-v2/train-00000-of-00001.parquet').to_dict(orient = 'records')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-1.7B-Base')
extra = [AddedToken('<|speech_start|>')]
for i in range(65536):
    extra.append(AddedToken(f'<|s_{i}|>'))
tokenizer.add_tokens(extra)

In [ ]:
import gc

def collator(batch, batch_position_ids):
    input_ids = []
    position_ids = []
    masks = []
    for i in range(len(batch)):
        l = len(batch[i])
        input_ids.extend(batch[i])
        position_ids.extend(batch_position_ids[i])
        masks.append(l)
    
    return {
        'input_ids': np.array(input_ids).astype(np.uint32),
        'position_ids': np.array(position_ids).astype(np.uint32),
        'attention_mask': np.array(masks).astype(np.uint32),
        'audio': '',
        'text': '',
    }

def slice_and_balance(nested_list, size):
    first = []
    balance = []
    current_size = 0

    for sublist in nested_list:
        if current_size < size:
            remaining_space = size - current_size
            if len(sublist) <= remaining_space:
                first.append(sublist)
                current_size += len(sublist)
            else:
                first.append(sublist[:remaining_space])
                balance.append(sublist[remaining_space:])
                current_size = size
        else:
            balance.append(sublist)
    
    return first, balance

In [ ]:
!ls gfs

In [ ]:
!rm -rf gfs/01be5b33/tokenized-10k-qwen3

In [ ]:
import time

sequence_length = 1024 * 10
def loop(files, block_size = sequence_length):
    rows, index = files
    out_root = f'gfs/01be5b33/tokenized-10k-qwen3/tokenized-{index}'
    os.system(f'rm -rf {out_root}')
    count = 0
    temp = []
    position_ids = []
    last_block, last_position_block = None, None
    with MDSWriter(out=out_root, columns=columns, compression=None, hashes=hashes) as out:
        for row in tqdm(rows):

            text = row['text']
            
            try:
                with open(row['token_filename']) as fopen:
                    token = json.load(fopen)
            except:
                continue

            if len(text.split()) > len(token):
                continue

            left = row['speaker'] +': ' + text
            
            token = ''.join([f'<|s_{t}|>' for t in token])
            prompt = f'<|im_start|>{left}<|speech_start|>{token}<|im_end|>'
            
            outputs = tokenizer(prompt, add_special_tokens = False)
            position = range(len(outputs['input_ids']))
            length = len(outputs['input_ids'])
            
            if count + length > block_size:
                o = collator(temp, position_ids)
                if o['input_ids'].shape[0] > 0:
                    out.write(o)
                temp = [outputs['input_ids']]
                position_ids = [position]
                count = length
                
            else:
                temp.append(outputs['input_ids'])
                position_ids.append(range(len(outputs['input_ids'])))
                count += len(outputs['input_ids'])
        
        if len(temp):
            o = collator(temp, position_ids)
            if o['input_ids'].shape[0] > 0:
                out.write(o)
            

In [ ]:
combined = data + malaysian

In [ ]:
multiprocessing(combined, loop, cores = 40, returned = False)

In [4]:
folders = sorted(glob('gfs/01be5b33/tokenized-10k-qwen3/tokenized-*'), key = lambda x: int(x.split('-')[-1]))
folders

['gfs/01be5b33/tokenized-10k-qwen3/tokenized-0',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-1',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-2',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-3',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-4',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-5',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-6',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-7',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-8',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-9',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-10',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-11',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-12',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-13',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-14',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-15',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-16',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-17',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-18',
 'gfs/01be5b33/tokenized-10k-qwen3/tokenized-19',
 'gfs/01be

In [5]:
!rm -rf gfs/01be5b33/multipacking-10k-tts

In [6]:
with MDSWriter(out='gfs/01be5b33/multipacking-10k-tts', columns=columns, compression=None, hashes=hashes) as out:
    for f in folders:
        try:
            dataset = LocalDataset(local=f)
            for i in tqdm(range(len(dataset))):
                out.write(dataset[i])
        except Exception as e:
            print(e)
            pass

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 9300.01it/s]


In [7]:
dataset = LocalDataset('gfs/01be5b33/multipacking-10k-tts')
(len(dataset) * 10240) / 1e9

14.64338432

In [2]:
# !hf upload Scicom-intl/tts-multipacking-10k \
# gfs/01be5b33/multipacking-10k-tts --repo-type=dataset --private